In [1]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
import numpy as np
import pandas as pd
from pathlib import Path
import os

In [15]:
dicnpy='../dataCIC2017/'
dicdata='../Datasets/CIC2017/'

groupnames=["Tuesday","Wednesday","Thursday","Friday"]#"Monday",
attack_types={0:"BENIGN",1:"FTP-Patator", 2:"SSH-Patator", 3:"DoS GoldenEye", 4:"DoS Hulk", 5:"DoS Slowhttptest", 6:"DoS slowloris", 7:"Heartbleed", 
                  8:"Web Attack � Brute Force", 9:"Web Attack � XSS", 10:"Web Attack � Sql Injection", 11:"Infiltration",12:"Bot", 13:"DDoS", 14:"PortScan"} # "Web Attack ?Brute Force"
    # Web Attack ?Brute Force, Web Attack ?XSS, Web Attack ?Sql Injection 

# Data preprocessing

In [16]:
# there are many 0x data in 'sport' and 'dsport' and other columns, so tranfer them to numeric.
def str2num(inivalue):
    if isinstance(inivalue,int):
        return inivalue
    elif isinstance(inivalue,str):
        if inivalue.isnumeric():
            return int(inivalue)
        else:
            if inivalue.startswith('0x',0):
                return int(inivalue,16)
            elif inivalue=='-' or inivalue.strip()=='':
                return 0
            else:
                print(inivalue)
                return 0
    
    else:
        print(inivalue)
        return 0

In [17]:
'''
split data into group including benign, each type of attack.
capture benign group with numeric type and append save in npy file.
the target of transferting attack_cat to numeric is for multi labels classifciation
'''
def savegroup(data_num, gname):
    # capture each attack group in numeric type, save in the dic of train_group.
    group_types=data_num[" Label"].value_counts()
    print(group_types)
    for ind, ptype in group_types.items():
        single_group=data_num[data_num[" Label"]==ind].copy() 
            # save in npy file
        p=Path(dicnpy+gname+"-"+str(ind)+'.npy')
        if p.exists():
            os.remove(p)
        np.save(dicnpy+gname+"-"+str(ind)+'.npy',single_group.sample(frac=1))
              

In [ ]:
# start reading data and prepare npy data for training 
# read 8 csv files
files=["Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv","Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv","Friday-WorkingHours-Morning.pcap_ISCX.csv",
       "Monday-WorkingHours.pcap_ISCX.csv", "Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv", "Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv","Tuesday-WorkingHours.pcap_ISCX.csv","Wednesday-workingHours.pcap_ISCX.csv"]
for i in range(0, len(files)):
    filedic=dicdata+files[i]
    print(filedic)
    groupname=files[i].split("-",1)
    train_num=pd.read_csv(filedic, header=0,skip_blank_lines='true')  #these 4 CSV files have not header line.    
    train_num.fillna(0,inplace=True)
    
    for inde in range(0, len(attack_types)):
        train_num.loc[train_num[" Label"]==str(attack_types[inde]), " Label"]=inde
        
    # check out if columns be the numeric type. and transfer them. must be the end of  processing.
    for i in train_num.columns:
        try:
            np.asarray(train_num[i]).astype(np.float32)
        except ValueError:
            numseries=[]
            for ind ,cont in train_num.iterrows():
                numseries.append(str2num(cont[i]))
            train_num[i]=numseries

    # save in npy file according attack types.
    savegroup(train_num, groupname[0] )

    #break
print(train_num.shape)
print(groupname)

In [19]:
# prepare the npy data for testing; each day as a group including benign and attack data.
ptestbenign=Path(dicnpy+"test-0.npy") # the dictionary of all benign data from Tuesday to Friday
pallattack=Path(dicnpy+"15.npy") # the dictionary of all attack data
if ptestbenign.exists():
    os.remove(ptestbenign)
if pallattack.exists():
    os.remove(pallattack)

for gitem in groupnames:
    pday=Path(dicnpy+gitem+'.npy')
    if pday.exists():
        os.remove(pday)

    for root,dirs,files in os.walk(dicnpy):
        for each in files:
            if each.find("-")>0:    
                namelist=each.split("-",1)
                if gitem==namelist[0]:
                    # save data of each day in a single npy file
                    if pday.exists():
                        cont=np.load(dicnpy+each,allow_pickle=True)
                        origcont=np.load(pday,allow_pickle=True)
                        newcont=np.concatenate([origcont,cont],axis=0) 
                    else:
                        newcont=np.load(dicnpy+each,allow_pickle=True)
                    np.save(pday,newcont)
                    # save the benign data from Tuesday to Friday to the file of test-0.npy
                    if namelist[1].split(".",1)[0]==str(0):
                        if ptestbenign.exists():
                            cont=np.load(dicnpy+each,allow_pickle=True)
                            origcont=np.load(ptestbenign,allow_pickle=True)
                            newcont=np.concatenate([origcont,cont],axis=0) 
                        else:
                            newcont=np.load(dicnpy+each,allow_pickle=True)
                        np.save(ptestbenign,newcont)
                    # save all attack data to the file of 15.npy
                    else:
                        if pallattack.exists():
                            cont=np.load(dicnpy+each,allow_pickle=True)
                            origcont=np.load(pallattack,allow_pickle=True)
                            newcont=np.concatenate([origcont,cont],axis=0) 
                        else:
                            newcont=np.load(dicnpy+each,allow_pickle=True)
                        np.save(pallattack,newcont)

In [ ]:
# grouping data of each attack data and all benign data  from Tuesday to Friday
# following the first metrics in the paper
ptestbenign=Path(dicnpy+"test-0.npy") # the dictionary of all benign data from Tuesday to Friday
              
for attackindex in range(1,15):
    for root,dirs,files in os.walk(dicnpy):
        for each in files:
            if each.find("-")>0:  
                namelist=each.split("-",1)
                attackfile=namelist[1].split(".",1)
                if attackindex==int(attackfile[0]):
                    print(attackindex)
                    cont=np.load(dicnpy+each,allow_pickle=True)
                    origcont=np.load(ptestbenign,allow_pickle=True)
                    newcont=np.concatenate([origcont,cont],axis=0) 
                    print(newcont.shape)
                    p=Path(dicnpy+"test-"+str(attackindex)+'.npy')
                    if p.exists():
                        os.remove(p)
                    np.save(dicnpy+"test-"+str(attackindex)+'.npy',newcont)